### Toughts about Gradients

#### Calculating glocal step in TensorFlow and understanding it:  
**global_step**  
global_step refers to the number of batches seen by the graph. Every time a batch is provided, the weights are updated in the direction that minimizes the loss. global_step just keeps track of the number of batches seen so far. When it is passed in the **minimize()** argument list, the variable is increased by one. Have a look at **optimizer.minimize()**.

You can get the global_step value using:   
**tf.train.global_step()**   
**tf.train.get_global_step()**  
**tf.train.get_or_create_global_step()**

### Gradient Clipping and Capping in TensorFlow:  
**gradient clipping**
https://hackernoon.com/gradient-clipping-57f04f0adae
![title](img/gclip.png)
Understand the difference:  
**minimize** - computes gradients, clips/caps and then apply gradients  
**apply gradients** - only applies the gradients (W - dW)  
**minimize looks like**  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  
gvs = optimizer.compute_gradients(cost)  
**here -1 and 1 are min and max values under which the gradient should be clipped**  
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]  
train_op = optimizer.apply_gradients(capped_gvs)

### Different way of computing gradients in TensorFlow Eager API
There are 4 ways to automatically compute gradients when eager execution is enabled (actually, they also work in graph mode):  

**tf.GradientTape** context records computations so that you can call tfe.gradient() to get the gradients of any tensor computed while recording with regards to any trainable variable.  
**tfe.gradients_function()** takes a function (say f()) and returns a gradient function (say fg()) that can compute the gradients of the outputs of f() with regards to the parameters of f() (or a subset of them).  
**tfe.implicit_gradients()** is very similar but fg() computes the gradients of the outputs of  f() with regards to all trainable variables these outputs depend on.  
**tfe.implicit_value_and_gradients()** is almost identical but fg() also returns the output of the function f().  

Usually, in Machine Learning, you will want to compute the gradients of the loss with regards to the model parameters (ie. variables), and you will generally also be interested in the value of the loss itself. For this use case, the simplest and most efficient options are tf.GradientTape and tfe.implicit_value_and_gradients() (the other two options do not give you the value of the loss itself, so if you need it, it will require extra computations). I personally prefer tfe.implicit_value_and_gradients() when writing production code, and tf.GradientTape when experimenting in a Jupyter notebook.

### Vanishing Gradients
Vanishing Gradient Problem is a difficulty found in training certain Artificial Neural Networks with gradient based methods (e.g Back Propagation). In particular, this problem makes it really hard to learn and tune the parameters of the earlier layers in the network. This problem becomes worse as the number of layers in the architecture increases.

This is not a fundamental problem with neural networks - it's a problem with gradient based learning methods caused by certain activation functions. Let's try to intuitively understand the problem and the cause behind it.

Problem

Gradient based methods learn a parameter's value by understanding how a small change in the parameter's value will affect the network's output. If a change in the parameter's value causes very small change in the network's output - the network just can't learn the parameter effectively, which is a problem.

This is exactly what's happening in the vanishing gradient problem -- the gradients of the network's output with respect to the parameters in the early layers become extremely small. That's a fancy way of saying that even a large change in the value of parameters for the early layers doesn't have a big effect on the output. Let's try to understand when and why does this problem happen.

Cause

Vanishing gradient problem depends on the choice of the activation function. Many common activation functions (e.g sigmoid or tanh) 'squash' their input into a very small output range in a very non-linear fashion. For example, sigmoid maps the real number line onto a "small" range of [0, 1]. As a result, there are large regions of the input space which are mapped to an extremely small range. In these regions of the input space, even a large change in the input will produce a small change in the output - hence the gradient is small.

This becomes much worse when we stack multiple layers of such non-linearities on top of each other. For instance, first layer will map a large input region to a smaller output region, which will be mapped to an even smaller region by the second layer, which will be mapped to an even smaller region by the third layer and so on. As a result, even a large change in the parameters of the first layer doesn't change the output much.

We can avoid this problem by using activation functions which don't have this property of 'squashing' the input space into a small region. A popular choice is Rectified Linear Unit which maps x to max(0,x).

Hopefully, this helps you understand the problem of vanishing gradients. I'd also recommend reading along this iPython notebook which does a small experiment to understand and visualize this problem, as well as highlights the difference between the behavior of sigmoid and rectified linear units.

### Gradient Descent vs Stocahstic Gradient Descent
https://www.quora.com/Whats-the-difference-between-gradient-descent-and-stochastic-gradient-descent